# In this notebook 
We will optimize Adaptive Exponential Integrate and Fire Model and Izhikevich Model 
allen data against some Allen Experimental data.


In [1]:
from neuronunit.allenapi.allen_data_efel_features_opt import opt_setup, opt_setup_two, opt_exec, opt_to_model
from neuronunit.optimization.optimization_management import check_bin_vm15
from neuronunit.optimization.model_parameters import MODEL_PARAMS, BPO_PARAMS, to_bpo_param
from neuronunit.optimization.optimization_management import dtc_to_rheo,inject_and_plot_model
from bluepyopt.allenapi.allen_data_driven import opt_to_model
from utils import dask_map_function
import matplotlib.pyplot as plt
import numpy as np
from neuronunit.optimization.data_transport_container import DataTC
import efel
from jithub.models import model_classes
import matplotlib.pyplot as plt
import quantities as qt
from neuronunit.optimization.optimization_management import inject_model30 # TODO rename, elicit spikes.



ids = [ 324257146,
        325479788,
        476053392,
        623893177,
        623960880,
        482493761,
        471819401
       ]

specimen_id = ids[1]
efel.__file__
efel_list = list(efel.getFeatureNames());
cellmodel = "ADEXP"

/home/user/anaconda3/lib/python3.7/site-packages/cerberus/validator.py:1609: UserWarning: No validation schema is defined for the arguments of rule 'not_zero_obs_zscore'
  "'%s'" % method_name.split('_', 2)[-1]


ModuleNotFoundError: No module named 'neuronunit.allenapi.allen_data_efel_features_opt'

In [ ]:

#cellmodel = "ADEXP";
if cellmodel == "IZHI":
    model = model_classes.IzhiModel()
if cellmodel == "MAT":
    model = model_classes.MATModel()
if cellmodel == "ADEXP":
    model = model_classes.ADEXPModel()



specimen id 623960880 \\
{\small  
\url{http://celltypes.brain-map.org/mouse/experiment/electrophysiology/623960880}
specimen id 623893177 \\
\url{http://celltypes.brain-map.org/mouse/experiment/electrophysiology/623893177}
specimen id 482493761 \\
\url{http://celltypes.brain-map.org/mouse/experiment/electrophysiology/482493761}
specimen id 471819401 \\
\url{http://celltypes.brain-map.org/mouse/experiment/electrophysiology/471819401}


In [ ]:
specific_filter_list = ['ISI_log_slope_1.5x',
                        'mean_frequency_1.5x',
                        'adaptation_index2_1.5x',
                        'first_isi_1.5x',
                        'ISI_CV_1.5x',
                        'median_isi_1.5x',
                        'Spikecount_1.5x',
                        'all_ISI_values',
                        'ISI_values',
                        'time_to_first_spike',
                        'time_to_last_spike',
                        'time_to_second_spike',
                        'spike_times',
                        'AHP_depth_abs_1.5x',
                        'peak_voltage_1.5x',
                        'voltage_base_1.5x',
                        'voltage']
simple_yes_list = specific_filter_list
target_num_spikes = 8


In [ ]:

def ranged_select_model(model_type):
    dtc = DataTC()
    dtc.backend = model_type
    model = dtc.dtc_to_model()
    dtc.attrs = model._backend.default_attrs
    dtc.params = {k:np.mean(v) for k,v in MODEL_PARAMS[cellmodel].items()}
    model = dtc.dtc_to_model()
    return model


In [ ]:
def check_out_model(model):
    dtc = model.model_to_dtc()
    dtc = dtc_to_rheo(dtc)
    vm,plt,dtc = inject_and_plot_model(dtc,plotly=False)
    plt.show()
    print(dtc.attrs)
    print(dtc.rheobase)


Notice the Izhikevich model Needs a lot of computational
resources to optimize. Later we will see that a good fit is possible with a fraction of the resources in the Adexp model, the problem is that the Izhikevich model has a larger number of sensative parameters.

In [ ]:
from sciunit.scores import ZScore
model = ranged_select_model(cellmodel)
model.params
model.backend
suite, target_current, spk_count, cell_evaluator, simple_cell = opt_setup(specimen_id,
                                                              cellmodel,
                                                              target_num_spikes,
                                                              provided_model=model,
                                                              fixed_current=False)

#cell_evaluator,simple_cell = opt_setup_two(model,cellmodel, suite, nu_tests, target_current, spk_count,provided_model=model,score_type=ZScore)
NGEN = 100
MU = 20

mapping_funct = dask_map_function
final_pop, hall_of_fame, logs, hist = opt_exec(MU,NGEN,mapping_funct,cell_evaluator)

# 100 generations miniumum fitness 0.71

In [ ]:

opt,target = opt_to_model(hall_of_fame,cell_evaluator,suite, target_current, spk_count)

In [ ]:
best_ind = hall_of_fame[0]
fitnesses = cell_evaluator.evaluate_with_lists(best_ind)
fitnesses;

In [ ]:
best_ind

In [ ]:
obnames = [obj.name for obj in cell_evaluator.objectives]

for i,j in zip(fitnesses,obnames):
    print(i,j)

In [ ]:


gen_numbers = logs.select('gen')
min_fitness = logs.select('min')
max_fitness = logs.select('max')
avg_fitness = logs.select('avg')
plt.plot(gen_numbers, max_fitness, label='max fitness')
plt.plot(gen_numbers, avg_fitness, label='avg fitness')
plt.plot(gen_numbers, min_fitness, label='min fitness')

plt.plot(gen_numbers, min_fitness, label='min fitness')
plt.semilogy()
plt.xlabel('generation #')
plt.ylabel('score (# std)')
plt.legend()
plt.xlim(min(gen_numbers) - 1, max(gen_numbers) + 1)
plt.show()

In [ ]:
plt.plot(gen_numbers, max_fitness, label='max fitness')
plt.show()

In [ ]:


opt.vm15 = None
vm15,_,_,opt=inject_model30(opt,solve_for_current=target_current)
model = opt.dtc_to_model()
plt.plot(opt.vm15.times,opt.vm15)
opt.vm15
model.attrs
model.backend

In [ ]:
target_current

In [ ]:
target.vm15 = suite.traces['vm15']

plt.plot(target.vm15.times,target.vm15)
plt.show()


In [ ]:
target.vm15 = suite.traces['vm15']
check_bin_vm15(target,opt)
opt.attrs

In [ ]:
cellmodel = "IZHI"

model = ranged_select_model(cellmodel)

model.params
model.backend
suite, target_current, spk_count, cell_evaluator, simple_cell = opt_setup(specimen_id,
                                                              cellmodel,
                                                              target_num_spikes,
                                                              provided_model=model,
                                                              fixed_current=False)

NGEN = 100
MU = 100

mapping_funct = dask_map_function
final_pop, hall_of_fame, logs, hist = opt_exec(MU,NGEN,mapping_funct,cell_evaluator)

In [ ]:
opt,target = opt_to_model(hall_of_fame,cell_evaluator,suite, target_current, spk_count)
target.vm15 = suite.traces['vm15']
check_bin_vm15(target,opt)
opt.attrs

# TODO make a nested Genetic Algorithm where the outer loop explores different preferred currents.

This will get rid of the oscillations.